In [ ]:
import pandas as pd
import numpy as np
import random
import itertools

import os
import json
import copy
import time
import matplotlib.pyplot as plt
from datetime import datetime as dt

import importlib
import gmm_generator as gg
import ts_clustering as tc
import dynamic_model_selection as dms
import descriptional_dimension as ddim
import ddim_change_symptom.ddim_simulation as simDdim
import fixed_share as fs
from IPython.display import display


## single change

In [ ]:
%%time

alpha_list = [0.2,0.5,1.0,2.0]
# alpha_list = [1.0]
num_sim = 10
eval_list = ['benefit_diff','benefit_th','benefit_sdms','benefit_fs','benefit_ws_fs','benefit_ws_fs_th',
                    'delay_diff','delay_th','delay_sdms','delay_fs','delay_ws_fs','delay_ws_fs_th']

eval_list.append('alpha')

res = pd.DataFrame([],columns=eval_list)
for a in alpha_list:
    for i in range(num_sim):
        res_tmp = pd.DataFrame(np.zeros((1,len(eval_list))),columns=eval_list)
        res_tmp['alpha'] = a
        sim_obj = simDdim.simDdim(alpha=a,random_state=i,conf_file='ddim_paramset_single.json',c_pattern='single')
        sim_obj.simulate_algorithm()
        for k,v in sim_obj.out['result']['evaluation'].items():
            if k in eval_list:
                res_tmp.loc[0,k] = v[0]
        
        res = pd.concat([res,res_tmp],axis=0)
            
        if i == 0:
            sim_obj.create_graph(type_name='single',tofile=True,file_format='png')
    
display(res)


## multiple change

In [ ]:
%%time

alpha_list = [0.2,0.5,1.0,2.0]
# alpha_list = [1.0]
num_sim = 10
eval_list = ['benefit_diff','benefit_diff_2','benefit_th','benefit_th_2','benefit_sdms','benefit_sdms_2',
                    'benefit_fs','benefit_fs_2','benefit_ws_fs','benefit_ws_fs_2','benefit_ws_fs_th','benefit_ws_fs_th_2',
                    'delay_diff','delay_diff_2','delay_th','delay_th_2','delay_sdms','delay_sdms_2',
                    'delay_fs','delay_fs_2','delay_ws_fs','delay_ws_fs_2','delay_ws_fs_th','delay_ws_fs_th_2']

eval_list.append('alpha')

res_multi = pd.DataFrame([],columns=eval_list)
for a in alpha_list:
    for i in range(num_sim):
        res_tmp = pd.DataFrame(np.zeros((1,len(eval_list))),columns=eval_list)
        res_tmp['alpha'] = a
        sim_obj = simDdim.simDdim(alpha=a,random_state=i,conf_file='ddim_paramset_multi.json',c_pattern='multi')
        sim_obj.simulate_algorithm()
        for k,v in sim_obj.out['result']['evaluation'].items():
            if k in eval_list:
                res_tmp.loc[0,k] = v[0]
                res_tmp.loc[0,'%s_2'%k] = v[1]
        
        res_multi = pd.concat([res_multi,res_tmp],axis=0)
            
        if i == 0:
            sim_obj.create_graph(type_name='multi',tofile=True,file_format='png')
    
display(res_multi)


In [ ]:
eval_list = ['benefit_diff','benefit_th','benefit_sdms','benefit_fs','benefit_ws_fs','benefit_ws_fs_th',
                    'delay_diff','delay_th','delay_sdms','delay_fs','delay_ws_fs','delay_ws_fs_th']
display(
    res.groupby('alpha')[eval_list].mean().reset_index()
)
eval_multi_list = ['benefit_diff','benefit_diff_2','benefit_th','benefit_th_2','benefit_sdms','benefit_sdms_2',
                                'benefit_fs','benefit_fs_2','benefit_ws_fs','benefit_ws_fs_2','benefit_ws_fs_th','benefit_ws_fs_th_2',
                                'delay_diff','delay_diff_2','delay_th','delay_th_2','delay_sdms','delay_sdms_2',
                                'delay_fs','delay_fs_2','delay_ws_fs','delay_ws_fs_2','delay_ws_fs_th','delay_ws_fs_th_2']
display(
    res_multi.groupby('alpha')[eval_multi_list].mean().reset_index()
)


In [ ]:

now_time = dt.now()
now_str = now_time.strftime("%Y%m%d-%H%M%S")
date_str = now_time.strftime("%Y%m%d")

res.to_csv('./data/out/ddim_single_%s/ddim_single_benefit_%s.csv'%(date_str,now_str))
res_multi.to_csv('./data/out/ddim_multi_%s/ddim_multi_benefit_%s.csv'%(date_str,now_str))

res.groupby('alpha')[eval_list].mean().reset_index().to_csv('./data/out/ddim_single_%s/ddim_single_benefit_summary_%s.csv'%(date_str,now_str))
res_multi.groupby('alpha')[eval_multi_list].mean().reset_index().to_csv('./data/out/ddim_multi_%s/ddim_multi_benefit_summary_%s.csv'%(date_str,now_str))


In [ ]:
res_summary_single = res.groupby('alpha')[eval_list].mean().reset_index()
res_summary_multi = res_multi.groupby('alpha')[eval_multi_list].mean().reset_index()
